In [14]:
import pandas as pd
import json
from jinja2 import Environment, FileSystemLoader
from selenium import webdriver
import os

## Load Latest Data to Dictionary (for simple code access reasons)

In [15]:
# Get Data from csv, filter for Stirling Council, strip uneeded columns and convert Changes Json from String to Dictionary
LatestData = pd.read_csv('Data Files//Latest Values.csv', converters={'Changes': json.loads, 'Previous_Row':json.loads}).query(f'Local_Authority == "Stirling"')[['Code','Period','Real_Value','FamilyRank','Previous_Row','Changes']]

# Get indicator info from csv and merge titles into LatestData
Info = pd.read_csv('Data Files//Indicator Information.csv')
LatestData = LatestData.merge(Info[['Code_Sortable', 'Title', 'Format_Python', 'AdditionalAxisDenominator_Python',
                              'SubGroup_PythonReport', 'ISCategory']], how='left', left_on=['Code'], right_on=['Code_Sortable'], suffixes=('_latest', '_info'))

# Add additional column that cotains formatted value strings
def formatvalue (df) : 
    df['Real_Value_Formatted'] = df['Format_Python'].format(df['Real_Value']/df['AdditionalAxisDenominator_Python'])
    return df['Real_Value_Formatted']
                                                      
LatestData['Real_Value_Formatted'] = LatestData.apply(formatvalue, axis = 1)

# Add additional column containing slice of Family Rank data for each indicator.
AllData = pd.read_csv('Data Files//Indicator Data.csv', converters={'Changes': json.loads}).query(f'Local_Authority == "Stirling"')[['Code','Period','FamilyRank','Real_Value']]
AllData = AllData.merge(Info[['Code_Sortable', 'Format_Python', 'AdditionalAxisDenominator_Python',]], how='left', left_on=['Code'], right_on=['Code_Sortable'], suffixes=('_latest', '_info'))
AllData['Real_Value_Formatted'] = AllData.apply(formatvalue, axis = 1)
fgtimeseries = []
valuetimeseries = []
for row in LatestData.itertuples() :
    Code = '"' + row.Code + '"'
    fgtimeseries.append(AllData.query(f'Code == {Code}')[['Period', 'FamilyRank']].set_index('Period').to_dict()['FamilyRank'])
    valuetimeseries.append(AllData.query(f'Code == {Code}')[['Period', 'Real_Value_Formatted']].set_index('Period').to_dict()['Real_Value_Formatted'])
    
results = []
for idx, item in enumerate(fgtimeseries) :
    result = {}
    for key in (item | valuetimeseries[idx]):
        if key in item: result.setdefault(key, []).append(item[key])
        if key in valuetimeseries[idx]: result.setdefault(key, []).append(valuetimeseries[idx][key])
    results.append(result)
        
results
LatestData['timeseries'] = results

# Set the index of the dataframe to the code and then convert to a dictionary
LatestDataDict = LatestData.set_index('Code').to_dict(orient="index")

# Data for select indicators can now be accessed through dictionary keys as below
LatestDataDict['C&L 01']


{'Period': '2020-21',
 'Real_Value': 213.35156342884315,
 'FamilyRank': 8.0,
 'Previous_Row': {'Real_Value': 2.422236746010734,
  'Real_Numerator': 2631527.689813045,
  'Real_Denominator': 1086404.0,
  'Cash_Value': 2.270794290153571,
  'Cash_Numerator': 2467000.0,
  'Cash_Denominator': 1086404.0,
  'ScotRank': 13.0,
  'ScotPct': 0.40625,
  'FamilyRank': 4.0,
  'FamilyPct': 0.5},
 'Changes': {'ScotRank_ChangeSincePrevious': -18.0,
  'ScotPct_ChangeSincePrevious': -0.5625,
  'FamilyRank_ChangeSincePrevious': -4.0,
  'FamilyPct_ChangeSincePrevious': -0.5,
  'ScotRank_ChangeSinceFirst': -21.0,
  'ScotPct_ChangeSinceFirst': -0.65625,
  'FamilyRank_ChangeSinceFirst': -5.0,
  'FamilyPct_ChangeSinceFirst': -0.625,
  'Real_Value_ChangeSincePrevious': 210.92932668283242,
  'Real_Numerator_ChangeSincePrevious': 957472.3101869547,
  'Real_Denominator_ChangeSincePrevious': -1069582.0,
  'Cash_Value_ChangeSincePrevious': 211.08076913868962,
  'Cash_Numerator_ChangeSincePrevious': 1122000.0,
  'Cash

### Load Latest Data Grouped by Groups and SubGroups into Dictionary for Use When Creating Overview Pages

In [16]:
# Define function to retrieve a specified nested value from the LatestDataDict
def getValueFromDict(Code, ParentField, Field) :
    return LatestDataDict.get(Code).get(ParentField).get(Field)

# Copy latestdata with relevant columns
GroupedLatestData = LatestData[['ISCategory','SubGroup_PythonReport','Code_Sortable','Title','Period','Real_Value_Formatted','Format_Python','AdditionalAxisDenominator_Python','FamilyRank']].copy(deep=True)

# Loop nested list that defines field extraction from dictionary required and add each as a separate column
FieldsRequired = [
    ['Changes','Real_Value_ChangeSincePrevious'],
    ['Changes','PercentChange_AimAdjusted_SincePrevious'],
    ['Changes','FamilyRank_ChangeSincePrevious'],
    ['Previous_Row','Real_Value'],
    ['Previous_Row','FamilyRank']
    ]
for i in FieldsRequired :
    GroupedLatestData[f'{i[0]}_{i[1]}'] = GroupedLatestData.apply(lambda df: getValueFromDict(df['Code_Sortable'],i[0],i[1]), axis = 1)

# Format Previous Row Real Value
GroupedLatestData['Previous_Row_Real_Value'] = GroupedLatestData.apply(lambda df: df['Format_Python'].format(df['Previous_Row_Real_Value']/df['AdditionalAxisDenominator_Python']), axis = 1)

# Remove and reorder columns as required
GroupedLatestData = GroupedLatestData[[
    'ISCategory', 'SubGroup_PythonReport', 'Code_Sortable', 'Title', 'Period', 'Real_Value_Formatted', 'Previous_Row_Real_Value', 
    'Changes_Real_Value_ChangeSincePrevious', 'Changes_PercentChange_AimAdjusted_SincePrevious', 'FamilyRank', 'Previous_Row_FamilyRank','Changes_FamilyRank_ChangeSincePrevious']]

ISCategories = GroupedLatestData['ISCategory'].unique()
GroupedLatestDataDict = {}
for Category in ISCategories :
    GroupedLatestDataDict[f'{Category}'] = {}
    SubCategories = GroupedLatestData.query(f'ISCategory == "{Category}"')['SubGroup_PythonReport'].unique()
    for SubCategory in SubCategories :
        GroupedLatestDataDict[f'{Category}'][f'{SubCategory}'] = {}
        Indicators = GroupedLatestData.query(f'SubGroup_PythonReport == "{SubCategory}" & ISCategory == "{Category}"')['Code_Sortable'].unique()
        for Indicator in Indicators :
            GroupedLatestDataDict[f'{Category}'][f'{SubCategory}'][f'{Indicator}'] = list(GroupedLatestData.query(f'Code_Sortable == "{Indicator}"')[['Title', 'Period', 'Real_Value_Formatted', 'Previous_Row_Real_Value', 'Changes_Real_Value_ChangeSincePrevious', 'Changes_PercentChange_AimAdjusted_SincePrevious', 'FamilyRank', 'Previous_Row_FamilyRank','Changes_FamilyRank_ChangeSincePrevious']].to_dict('index').values())[0]
            
GroupedLatestDataDict

{'Culture & Leisure Services': {'Leisure Facilities & Attractions': {'C&L 01': {'Title': 'Cost per attendance at Sports facilities',
    'Period': '2020-21',
    'Real_Value_Formatted': '£ 213.35',
    'Previous_Row_Real_Value': '£ 2.42',
    'Changes_Real_Value_ChangeSincePrevious': 210.92932668283242,
    'Changes_PercentChange_AimAdjusted_SincePrevious': -87.08039254636,
    'FamilyRank': 8.0,
    'Previous_Row_FamilyRank': 4.0,
    'Changes_FamilyRank_ChangeSincePrevious': -4.0},
   'C&L 02': {'Title': 'Cost Per Library Visit',
    'Period': '2020-21',
    'Real_Value_Formatted': '£ 3.05',
    'Previous_Row_Real_Value': '£ 2.32',
    'Changes_Real_Value_ChangeSincePrevious': 0.7368775307529929,
    'Changes_PercentChange_AimAdjusted_SincePrevious': -0.3179655527973885,
    'FamilyRank': 4.0,
    'Previous_Row_FamilyRank': 5.0,
    'Changes_FamilyRank_ChangeSincePrevious': 1.0},
   'C&L 03': {'Title': 'Cost of Museums per Visit',
    'Period': '2020-21',
    'Real_Value_Formatted': 

In [17]:
ISCategories

array(['Culture & Leisure Services', "Children's Services",
       'Tackling Climate Change', 'Corporate Services',
       'Economic Development', 'Environmental Services',
       'Financial Sustainability', 'Housing Services',
       'Adult Social Care Services'], dtype=object)

### Setup Jinja2 Environment, Define CreateGroupPage and SavetoPDF functions and then create all pages from templates.

In [18]:
env = Environment(loader=FileSystemLoader('Templates_Jinja2'))

### Define Repeated Functions

In [19]:
def createGroupPage(pagename, layouttitle) :
    template = env.get_template(layouttitle)
    htmlout = template.render(LatestData = LatestDataDict)
    file = open("Pages_FinalReport/" + pagename, "w")
    file.write(htmlout)
    file.close()

def savetopdf (relativepath) :
    chrome_options = webdriver.ChromeOptions()
    #headless breaks the pdf outputting for some reason
    chrome_options.add_argument("--headless")
    
    #settings or prefs here need altered to specify A3, landscape, no header and footer and print css backgrounds. Ideally also download path so that changes are saved directly into a folder in the repo files.
    settings = {"recentDestinations": [{"id": "Save as PDF", "origin": "local", "account": ""}], "selectedDestinationId": "Save as PDF", "version": 2}
    prefs = {'printing.print_preview_sticky_settings.appState': json.dumps(settings)}
    chrome_options.add_experimental_option('prefs', prefs)
    chrome_options.add_argument('--kiosk-printing')
    
    #something here is depricated and requires replaced as per error output when used
    browser = webdriver.Chrome(r"C:\\Program Files\\Google\\Chrome\\Application\\chromedriver.exe", options=chrome_options)
    htmlfilepath = "file:///" +  os.path.abspath(relativepath)
    browser.get(htmlfilepath)
    browser.execute_script('window.print();')

### Generate Group Pages HTML

In [20]:
For file in folder :
    if title contains Layout_Group
    then createGroupPage(File.title)
    elif title contains Layout_Overview
    then create

createGroupPage("Group_AdultHealthCare.html","Layout_AdultHealthCare.html")
createGroupPage("Group_Roads.html","Layout_Roads.html")
createGroupPage("Group_AtHomeCare.html","Layout_AtHomeCare.html")
createGroupPage("Group_Waste.html","Layout_Waste.html")
createGroupPage("Group_StreetCleaning.html","Layout_Street Cleaning.html")
createGroupPage("Group_CostOfRegulation.html","Layout_Cost of Regulation.html")
createGroupPage("Group_Employment.html","Layout_Employment.html")
createGroupPage("Group_EarlyYears.html","Layout_Early Years.html")
createGroupPage("Group_SchoolAttendance.html","Layout_School Attendance.html")
createGroupPage("Group_PropertyandAssets.html", "Layout_PropertyandAssets.html")
createGroupPage("Group_Ab+Gen.html", "Layout_Ab+Gen.html")
createGroupPage("Group_Pa+Cl.html", "Layout_Pa+Cl.html")
#savetopdf('./Pages_FinalReport/Pa+Cl.html')

### Generate Overview Pages HTML

In [21]:
template = env.get_template("Layout_Overview_Page1.html")
htmlout = template.render(GroupedLatestDataDict = GroupedLatestDataDict)
file = open("Pages_FinalReport/Overview_Page1.html", "w")
file.write(htmlout)
file.close()

template = env.get_template("Layout_Overview_Page2.html")
htmlout = template.render(GroupedLatestDataDict=GroupedLatestDataDict)
file = open("Pages_FinalReport/Overview_Page2.html", "w")
file.write(htmlout)
file.close()

template = env.get_template("Layout_Overview_Page3.html")
htmlout = template.render(GroupedLatestDataDict=GroupedLatestDataDict)
file = open("Pages_FinalReport/Overview_Page3.html", "w")
file.write(htmlout)
file.close()

### Generate Front Cover HTML

In [22]:
template = env.get_template("Layout_FrontCover.html")
htmlout = template.render(ReportYear = "2020/21", ReportTitle = "Local Government Benchmark Framework", ReportTitleDescription = "Stirling Council by Theme", PublicationDate = "01/01/2022", ImgPath = "../Images/Stirling_unsplash.jpg")
file = open("Pages_FinalReport/FrontCover.html", "w")
file.write(htmlout)
file.close()